In [1]:
import pandas as pd
import numpy as np
import requests
import json

import pymysql
from sqlalchemy import create_engine
import sqlite3


In [2]:
url = "https://data.ny.gov/resource/q4hy-kbtf.json?$limit=500000"
headers = {
"Content-Type" : "application/json",
}


response_API = requests.get(url,headers=headers)
print(response_API.status_code)


200


In [3]:
data = response_API.text
data = json.loads(data)
data

[{'violation_charged_code': '1110A',
  'violation_description': 'DISOBEYED TRAFFIC DEVICE',
  'violation_year': '2019',
  'violation_month': '1',
  'violation_dow': 'WEDNESDAY',
  'age_at_violation': '34',
  'gender': 'M',
  'state_of_license': 'NEW YORK',
  'police_agency': 'NYC POLICE DEPT',
  'court': 'BROOKLYN SOUTH TVB',
  'source': 'TVB'},
 {'violation_charged_code': '1111D1N',
  'violation_description': 'NYC REDLIGHT',
  'violation_year': '2019',
  'violation_month': '1',
  'violation_dow': 'THURSDAY',
  'age_at_violation': '28',
  'gender': 'M',
  'state_of_license': 'NEW YORK',
  'police_agency': 'NYC POLICE DEPT',
  'court': 'BROOKLYN SOUTH TVB',
  'source': 'TVB'},
 {'violation_charged_code': '1163AT',
  'violation_description': 'IMPROPER TURN',
  'violation_year': '2019',
  'violation_month': '6',
  'violation_dow': 'SATURDAY',
  'age_at_violation': '57',
  'gender': 'M',
  'state_of_license': 'NEW JERSEY',
  'police_agency': 'NYC POLICE DEPT',
  'court': 'QUEENS SOUTH TVB'

In [4]:
len(data)

500000

### Filtering the data

In [5]:
parameters = {
    "violation_year":"2022",
    'violation_month': '2',
    'court': 'BROOKLYN SOUTH TVB'
}

url2 = "https://data.ny.gov/resource/q4hy-kbtf.json?$limit=500000"
headers = {
"Content-Type" : "application/json",
}


res = requests.get(url2,params=parameters)
print(res.status_code)


200


In [6]:
data2 = res.text
data2 = json.loads(data2)
data2

[{'violation_charged_code': '1131',
  'violation_description': 'DRIVING ON SHOULDER/LIMITED ACCESS HGWY',
  'violation_year': '2022',
  'violation_month': '2',
  'violation_dow': 'MONDAY',
  'age_at_violation': '53',
  'gender': 'M',
  'state_of_license': 'NEW YORK',
  'police_agency': 'NYC POLICE DEPT',
  'court': 'BROOKLYN SOUTH TVB',
  'source': 'TVB'},
 {'violation_charged_code': '1131',
  'violation_description': 'DRIVING ON SHOULDER/LIMITED ACCESS HGWY',
  'violation_year': '2022',
  'violation_month': '2',
  'violation_dow': 'MONDAY',
  'age_at_violation': '51',
  'gender': 'F',
  'state_of_license': 'NEW YORK',
  'police_agency': 'NYC POLICE DEPT',
  'court': 'BROOKLYN SOUTH TVB',
  'source': 'TVB'},
 {'violation_charged_code': '1110A',
  'violation_description': 'DISOBEYED TRAFFIC DEVICE',
  'violation_year': '2022',
  'violation_month': '2',
  'violation_dow': 'MONDAY',
  'age_at_violation': '51',
  'gender': 'M',
  'state_of_license': 'NEW YORK',
  'police_agency': 'NYC POLI

In [12]:
len(data2)

8442

### Creating dataframe

In [13]:
court_list = ["BRONX TVB","BROOKLYN NORTH TVB","BROOKLYN SOUTH TVB","MANHATTAN NORTH TVB","MANHATTAN SOUTH TVB",
              "QUEENS NORTH TVB","QUEENS SOUTH TVB","RICHMOND TVB"]


In [14]:
data_list = []
for month in range(1,13):
    for court in court_list:
        parameters = {"violation_year":"2022",'violation_month': str(month),'court': court}
        url2 = "https://data.ny.gov/resource/q4hy-kbtf.json?$limit=500000"
        res = requests.get(url2,params=parameters)
        number = len(json.loads(res.text))
        
        data_list.append([court, str(month), "2022", str(number)])
        
        
data = pd.DataFrame(data_list,columns = ["court","violation_month","violation_year","no_of_violations"])
data      

,court,violation_month,violation_year,no_of_violations
0,BRONX TVB,1,2022,6867
1,BROOKLYN NORTH TVB,1,2022,4021
2,BROOKLYN SOUTH TVB,1,2022,7359
3,MANHATTAN NORTH TVB,1,2022,4704
4,MANHATTAN SOUTH TVB,1,2022,5379
...,...,...,...,...
91,MANHATTAN NORTH TVB,12,2022,6139
92,MANHATTAN SOUTH TVB,12,2022,5234
93,QUEENS NORTH TVB,12,2022,5884
94,QUEENS SOUTH TVB,12,2022,5496


In [18]:
def split_court(col):
    return col.split(" ")[0]
    

In [19]:
data["court"] = data["court"].apply(split_court)
dataFrame = data.groupby(["court","violation_month","violation_year"]).sum().reset_index()
dataFrame

,court,violation_month,violation_year,no_of_violations
0,BRONX,1,2022,6867
1,BRONX,10,2022,8760
2,BRONX,11,2022,8621
3,BRONX,12,2022,7898
4,BRONX,2,2022,6684
5,BRONX,3,2022,8328
6,BRONX,4,2022,7519
7,BRONX,5,2022,7742
8,BRONX,6,2022,8205
9,BRONX,7,2022,8306


### Storing in MySQL

In [22]:
# Establish connection to AWS RDS database
host = 'database-3.csuqjb4hx3vr.us-east-1.rds.amazonaws.com'
user = 'admin'
password = 'jackisback'
db_name = 'CAR_CRASH_DATA'

In [26]:
conn = pymysql.connect(host=host, user=user, password=password, db=db_name)

try:
   # Create SQLAlchemy engine to connect to MySQL Database
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=host, db=db_name, user=user, pw=password))

    # Convert dataframe to sql table                                   
    dataFrame.to_sql('traffic_violations_data', engine, index=False)

except ValueError as vx:
    print(vx)
except Exception as ex:
    print(ex)
else:
    print("Table is created successfully.")
finally:
    conn.close()

Table is created successfully.
